In [ ]:
from daphne import daphne
import os

In [ ]:
directory = 'programs/tests/hoppl-deterministic'
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/{}/'.format(directory))

In [ ]:
i=3
cwd='/Users/gw/repos/daphne'
exp = daphne(['desugar-hoppl', '-i', '../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne'.format(i)])
exp



In [ ]:
desugared_ast_json_fname = '/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne.json'.format(i)
with open(desugared_ast_json_fname, 'w') as f:
    json.dump(exp, f)

In [ ]:
%ls /Users/gw/repos/daphne/../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic

In [ ]:
class Env():
    "An environment: a dict of {'var': val} pairs, with an outer Env."
    def __init__(self, parms=(), args=(), outer=None):
        self.data = pmap(zip(parms, args))
        self.outer = outer
        if outer is None:
            self.level = 0
        else:
            self.level = outer.level+1
    def __getitem__(self, item):
        return self.data[item]
    def find(self, var):
        "Find the innermost Env where var appears."
        if (var in self.data):
            return self
        else:
            if self.outer is not None:
                return self.outer.find(var)
            else:
                raise RuntimeError('var "{}" not found in outermost scope'.format(var))
    
    def print_env(self, print_lowest=False):
        print_limit = 1 if print_lowest == False else 0
        outer = self
        while outer is not None:
            if outer.level >= print_limit:
                print('Scope on level ', outer.level)
                if 'f' in outer:
                    print('Found f, ')
                    print(outer['f'].body)
                    print(outer['f'].parms)
                    print(outer['f'].env)
                print(outer,'\n')
            outer = outer.outer

In [ ]:
import copy

class Procedure(object):
    "A user-defined Scheme procedure."
    def __init__(self, parms, body, env):
        self.parms, self.body, self.env = parms, body, env
    def __call__(self, *args): 
        new_env = copy.deepcopy(self.env)
        return eval(self.body, Env(self.parms, args, new_env)) # [0]

In [ ]:
from pyrsistent import pmap

import torch
import torch.distributions as dist

In [ ]:
def push_addr(alpha, value):
    return alpha + value

penv = {
    '+': torch.add,
    '*':torch.mul,
#     'push-address':push_addr
}

In [ ]:
def standard_env():
    "An environment with some Scheme standard procedures."
    env = Env(penv.keys(), penv.values())
    return env

env = standard_env()
env.find('+')
#['+'](torch.tensor(0.),torch.tensor(1.))

In [ ]:
def eval(x,env=standard_env(),sigma=None):

    print('x',x)
    if isinstance(x,str):
        return env.find(x)[x], sigma
    elif not isinstance(x,list):
        return torch.tensor(x),sigma
    
    op, param, *args = x
    
    if False:
        pass
    
    elif op == 'push-address':
        return '', sigma
    elif op == 'fn':
        print('args',args)
#         param, body = args
        body = args[0]
        return Procedure(param, body, env), sigma # has eval in it
        # param ['alpha', 'x']
        # body ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']
        # env
    
    else:
        print('in else. x',x)
        proc, _ = eval(op,env,sigma)
        vals = [''] # TODO: address for fn???
        vals.extend([eval(arg, env, sigma)[0] for arg in args])
        print('vals',vals)

        if isinstance(proc, Procedure): # lambdas, not primitives
            print('in Procedure',proc, vals)
            r, _ = proc(*vals)
        else:
            r = proc(*vals[1:]) # primitives
            print('in primitives',proc, vals)
            
        return r, sigma
    

# eval(['*','a0',3,3])  

# why vals only tensor(8.), where did push add go?

In [ ]:
exp

In [ ]:
proc, sigma = eval(exp)
proc('')

In [ ]:
exp_sq_8 = [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']],
   ['push-address', 'alpha', 'addr1'],
   9]

eval(exp_sq_8)#(torch.tensor(8.))

In [ ]:
exp_inner = ['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
eval(exp_inner)#(torch.tensor(8.))

# test driven development

In [ ]:
import sys
sys.setrecursionlimit(10000)

## cons

In [91]:
exp = [tensor(0.), tensor([1,2,3])]

torch.cat((torch.tensor([exp[0]]), exp[1]), dim=0)

tensor([0., 1., 2., 3.])

In [ ]:
def cons(*exp):
    try:
        return torch.cat((torch.tensor([exp[1]]), exp[0]), dim=0)
    except:
        return torch.cat((torch.tensor([exp[1]]), torch.tensor(exp[0])), dim = 0)

## hash-map

In [ ]:
# eval_hoppl(['hash-map','','a',1,'b',2])

In [ ]:
# from old_primitives import penv
# from torch import tensor

# vals = ['',tensor(1.),tensor(2.)]
# penv['+'](vals[1:])

In [ ]:
# fname = 'programs/tests/hoppl-deterministic/test_{}.daphne.json'.format(i)

# with open(fname) as f:
#      exp = json.load(f)
# exp
# # 

## empty

In [9]:
from primitives import penv
penv['empty?'](tensor([]))

True

In [ ]:
['', tensor([])][1:]

In [10]:
eval_hoppl(['empty?','',['vector','']],do_log=True)

x ['empty?', '', ['vector', '']]
case else: x ['empty?', '', ['vector', '']]
x empty?
case else: args [['vector', '']]
x ['vector', '']
case else: x ['vector', '']
x vector
case else: args []
case else: vals ['']
case Procedure: <function vector_primitive at 0x139fe1160> ['']
case primitives: r tensor([])
case else: vals ['', tensor([])]
case Procedure: <function empty_primitive at 0x139fec550> ['', tensor([])]
case primitives: r False


(False, None)

In [ ]:
args=[[tensor(0.), tensor(1.)]]

[eval_hoppl(arg)[0] for arg in args]

In [ ]:
eval_hoppl(['empty?','',[tensor(0.),tensor(1.)]],do_log=True)

In [23]:
eval_hoppl(['vector','',tensor(1),tensor(2)])

(tensor([1, 2]), None)

## deterministic-hoppl tests

In [97]:
from tests import load_truth, is_tol
import os, json
from daphne import daphne
import torch
from torch import tensor

import importlib
import evaluator
importlib.reload(evaluator)
from evaluator import evaluate, run_deterministic_tests, ast_helper, eval_hoppl

import primitives
importlib.reload(primitives)

<module 'primitives' from '/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/primitives.py'>

In [95]:
i='cons'
fname='test_{}.daphne'.format(i)
exp = ast_helper(fname,directory='programs/tests/deterministic')
exp

['fn',
 ['alpha'],
 ['cons',
  ['push-address', 'alpha', 'addr0'],
  1,
  ['vector', ['push-address', 'alpha', 'addr1'], 2, 3, 4]]]

In [88]:
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/')
truth = load_truth('programs/tests/hoppl-deterministic/test_{}.truth'.format(i))
truth

tensor([2, 3, 4])

In [98]:
ret = evaluate(exp, do_log=True)
r
# assert(is_tol(ret, truth))

# 4,6,8 quote a and b. daphne output good?

x ['fn', ['alpha'], ['cons', ['push-address', 'alpha', 'addr0'], 1, ['vector', ['push-address', 'alpha', 'addr1'], 2, 3, 4]]]
case fn: args [['cons', ['push-address', 'alpha', 'addr0'], 1, ['vector', ['push-address', 'alpha', 'addr1'], 2, 3, 4]]]
x ['cons', ['push-address', 'alpha', 'addr0'], 1, ['vector', ['push-address', 'alpha', 'addr1'], 2, 3, 4]]
case else: x ['cons', ['push-address', 'alpha', 'addr0'], 1, ['vector', ['push-address', 'alpha', 'addr1'], 2, 3, 4]]
x cons
case else: args [1, ['vector', ['push-address', 'alpha', 'addr1'], 2, 3, 4]]
x 1
x ['vector', ['push-address', 'alpha', 'addr1'], 2, 3, 4]
case else: x ['vector', ['push-address', 'alpha', 'addr1'], 2, 3, 4]
x vector
case else: args [2, 3, 4]
x 2
x 3
x 4
case else: vals ['', tensor(2), tensor(3), tensor(4)]
case Procedure: <function vector_primitive at 0x13a8264c0> ['', tensor(2), tensor(3), tensor(4)]
case primitives: r tensor([2, 3, 4])
case else: vals ['', tensor(1), tensor([2, 3, 4])]
case Procedure: <function c

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

## all tests

In [ ]:
%cd /Users/gw/repos/prob_prog/hw/hw5/CS532-HW5
run_deterministic_tests()